In [ ]:
import sys # for automation and parallelisation
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])
if manual:
    %matplotlib inline

In [1]:
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.messaging as message
from biogeme import expressions as ex
import pandas as pd
import numpy as np
import xlsxwriter

# Calibration
## Estimation of the model parameters
quetzal_germany is being estimated using [PandasBiogeme](https://biogeme.epfl.ch/). This notebook estimates calibration parameters for the model's utility functions.
- Documentation and reference: [Bierlaire, M. (2020). A short introduction to PandasBiogeme. Technical report TRANSP-OR 200605. Transport and Mobility Laboratory, ENAC, EPFL.](https://transp-or.epfl.ch/documents/technicalReports/Bier20.pdf)
- Tutorial: https://www.youtube.com/watch?v=OiM94B8WayA

### Model formulation
The model consists of systematic utility functions, one for each mode. They comprise an alternaive-specific constant (ASC), a distance-dependent part with travel time and cost summarised as generalised cost (GC), and a cost damping function F

$V_i = ASC + F(GC(T, C), b_{gc_i})$

Index i marks the demand group. I = {'commuting' (1), 'education' (2), 'shopping/medical' (3), 'business' (4), 'private' (6)}

Note: The cost variable already includes subscriptions

In [2]:
input_path = '../input/'
model_path = '../model/'

### Prepare the database

In [3]:
df = pd.read_csv(input_path + 'transport_demand/calibration_inter-cellular_trips_MiD2017.csv')
df = df[['cost_rail_short', 'cost_rail_long', 'cost_car', 'cost_coach', 'cost_bus', 'cost_walk', 'cost_air',
         'time_rail_short', 'time_rail_long', 'time_car', 'time_coach', 'time_bus', 'time_walk', 'time_air',
         'cost_rail', 'cost_road', 'time_rail', 'time_road',
#         'accessibility_rail', 'accessibility_car', 'accessibility_coach', 'accessibility_bus',
#         'accessibility_walk', 'accessibility_air',
         'mode_model', 'purpose_vp', 'car_avail', 'length']]
df.columns = ['C_RAIL_S', 'C_RAIL_L', 'C_CAR', 'C_COACH', 'C_BUS', 'C_NON_MOTOR', 'C_AIR',
              'T_RAIL_S', 'T_RAIL_L', 'T_CAR', 'T_COACH', 'T_BUS', 'T_NON_MOTOR', 'T_AIR',
              'C_RAIL', 'C_ROAD', 'T_RAIL', 'T_ROAD',
#              'AC_RAIL', 'AC_CAR', 'AC_COACH', 'AC_BUS', 'AC_NON_MOTOR', 'AC_AIR',
              'MODE', 'PURPOSE', 'CAR_AV', 'DIST']

In [4]:
inf = 1000
df = df.replace({np.inf:inf})

In [5]:
df.describe()

,C_RAIL_S,C_RAIL_L,C_CAR,C_COACH,C_BUS,C_NON_MOTOR,C_AIR,T_RAIL_S,T_RAIL_L,T_CAR,...,T_NON_MOTOR,T_AIR,C_RAIL,C_ROAD,T_RAIL,T_ROAD,MODE,PURPOSE,CAR_AV,DIST
count,95309.000000,95309.000000,95309.000000,95309.000000,95309.000000,95309.0,95309.000000,95309.000000,95309.000000,95309.000000,...,95309.000000,95309.000000,95309.000000,95309.000000,95309.000000,95309.000000,95309.000000,95309.000000,95309.000000,95309.000000
mean,14.923703,82.591930,9.164346,7.640851,3.246669,0.0,950.651073,137.217725,576.536810,83.677637,...,453.220405,960.865623,17.000884,5.594604,106.130224,210.289958,5.553390,3.654702,0.964410,97.138377
std,13.022019,56.578643,12.283995,6.814230,1.275041,0.0,202.770843,189.395408,425.568844,88.400053,...,450.099583,161.739808,17.230333,5.700437,99.225155,257.640411,1.352007,2.059519,0.189299,136.947231
min,0.000000,19.000000,0.169506,5.000000,0.000000,0.0,35.000000,23.733333,29.950000,10.383333,...,2.660412,4.328056,0.000000,0.000000,13.000000,20.916667,1.000000,1.000000,0.000000,0.265000
25%,6.236112,19.483979,2.926655,5.000000,2.740000,0.0,1000.000000,49.516667,125.483333,37.166667,...,76.333333,1000.000000,6.971693,4.000000,49.100000,59.716667,6.000000,1.000000,1.000000,29.948000
50%,10.997933,72.207529,5.135950,5.000000,4.000000,0.0,1000.000000,70.466667,469.166667,52.033333,...,120.150000,1000.000000,11.803414,4.000000,68.891667,104.250000,6.000000,3.000000,1.000000,45.640000
75%,18.071347,139.000000,9.219859,5.000000,4.000000,0.0,1000.000000,120.000000,1000.000000,79.033333,...,1000.000000,1000.000000,19.334440,5.000000,115.733333,231.658333,6.000000,6.000000,1.000000,82.132000
max,42.000000,139.000000,119.704134,59.152434,4.000000,0.0,1000.000000,1000.000000,1000.000000,705.650000,...,1000.000000,1000.000000,139.000000,45.187855,1000.000000,1000.000000,7.000000,6.000000,9.000000,1045.096000


In [6]:
# Scale time to hours
df['T_RAIL_Sc'] = df['T_RAIL'] / 60
df['T_RAIL_S_S'] = df['T_RAIL_S'] / 60
df['T_RAIL_L_S'] = df['T_RAIL_L'] / 60
df['T_CAR_S'] = df['T_CAR'] / 60
df['T_COACH_S'] = df['T_COACH'] / 60
df['T_BUS_S'] = df['T_BUS'] / 60
df['T_ROAD_S'] = df['T_ROAD'] / 60
df['T_AIR_S'] = df['T_AIR'] / 60
df['T_NON_MOTOR_S'] = df['T_NON_MOTOR'] / 60

In [7]:
# Make car availability binary
df['CAR_AV'] = df['CAR_AV'].replace({9:0})

In [8]:
# Add PT availabilities
df['RAIL_AV'] = (df['T_RAIL']!=inf).astype(int)
df['RAIL_S_AV'] = (df['T_RAIL_S']!=inf).astype(int)
df['RAIL_L_AV'] = (df['T_RAIL_L']!=inf).astype(int)
df['COACH_AV'] = (df['T_COACH']!=inf).astype(int)
df['BUS_AV'] = (df['T_BUS']!=inf).astype(int)
df['ROAD_AV'] = (df['T_ROAD']!=inf).astype(int)
df['AIR_AV'] = (df['T_AIR']!=inf).astype(int)
df['NON_MOTOR_AV'] = (df['C_NON_MOTOR']!=inf).astype(int)

In [9]:
# Merge rail and road PT
df['MODE'] = df['MODE'].replace({2:1, 3:4})
len(df.loc[df['MODE'].isin([2,3])])

0

In [10]:
# Remove trips where mode is car but the car availability is zero
# because it irritates the MLE algorithm
mask = ((df['MODE']==6) & (df['CAR_AV']==0))
drop_ratio = len(df.loc[mask])/len(df.loc[df['MODE']==6])
lengths = list(df.loc[mask, 'DIST'])
print('Share of car trips dropped: {}'.format(drop_ratio))
df = df.loc[~mask]
# Drop other modes equally
for m in [1,4]:
    n_drops = int(np.round(drop_ratio * len(df.loc[df['MODE']==m])))
    df = df.drop(df.loc[(df['MODE']==m) & (df['DIST']<max(lengths)) &
                        (df['DIST']>min(lengths))].sample(n_drops).index)
    print('Dropped ' + str(n_drops) + ' trips of mode ' + str(m))
print(len(df))

Share of car trips dropped: 0.014148505211025496
Dropped 106trips of mode 1
Dropped 50trips of mode 4
93976


In [11]:
# Check that PT trips have no infinity cost
for mode, col in zip([1, 4, 5], ['RAIL_AV', 'ROAD_AV', 'AIR_AV']):
    assert len(df.loc[((df['MODE']==mode) & ~(df[col]))].index) == 0, str(mode)

### Model variables
All columns are variables. DefineVariable creates a new column in the database.

#### Cost damping

Many modelling studies have shown that cost damping is required in order to flatten the tail of time/cost elasticities, i.e. decrease the impact of long distances on choice results to prevent from overestimation of time/cost parameters. Cost damping represents the property of decreasing marginal utility. It is commonly approached with Box-Cox transformations of generalised cost (usually defined as the sum of travel time and travel expenditures divided by the value of time). Daly (2010) proposes a hybrid function as a sum of the linear term and the narural logarithm of the same. However, the linear term still dominates cost on long distances. Rich (2015), main developer of the Danish National Transport Model, proposes a more complex spline function which successfully manages cost damping and even outperforms the Box-Cox transformation in terms of stability of elasticities.

In [13]:
# Using the hybrid linear-logarithmic version requires a new column
# for the logarithm of time in minutes
'''df['T_RAIL_S'] = np.log(df['T_RAIL'])
df['T_CAR_S'] = np.log(df['T_CAR'])
df['T_COACH_S'] = np.log(df['T_COACH'])
df['T_BUS_S'] = np.log(df['T_BUS'])
df['T_AIR_S'] = np.log(df['T_AIR'])
df['T_NON_MOTOR_S'] = np.log(df['T_NON_MOTOR'])'''
print('This functional form performes poorly')

This functional form performes poorly


#### Generalised cost

Both, travel time and monetary cost should be affected by cost damping measures. It is logical to define a generalised cost term `GC` with dimension of time units. This requires definition or estimation of values of time, in order to rescale monetary units, for all segments. Usually, the value of time (VoT) is distance-dependent. In the case of Germany, VoT can be taken from research undertaken within the Federal Government's transport study "Bundesverkehrswegeplan 2030": Axhausen et al. 2015. Ermittlung von Bewertungsansätzen für Reisezeiten und Zuverlässigkeit auf der Basis eines Modells für modale Verlagerungen im nicht-gewerblichen und gewerblichen Personenverkehr für die Bundesverkehrswegeplanung

In [12]:
# VoT from literature, distance-dependent, see cal19
VoT = pd.read_csv(input_path + 'vot.csv', header=[0,1], index_col=0)
VoT.sample(2)

,root,Fz1,Fz2,Fz3,Fz4,Fz6,root,Fz1,Fz2,Fz3,...,Fz2,Fz3,Fz4,Fz6,root,Fz1,Fz2,Fz3,Fz4,Fz6
,all,all,all,all,all,all,PT,PT,PT,PT,...,air,air,air,air,car,car,car,car,car,car
582,19.0434,18.2378,9.18,11.9,24.0204,14.4762,14.3818,11.1112,13.48,15.5412,...,59.2990,59.2990,125.2246,33.9308,18.8606,17.9286,11.3558,23.9538,20.6730,14.4762
989,24.6193,23.4881,9.18,11.9,35.0908,18.1799,18.4111,13.7974,13.48,18.2274,...,105.9005,105.9005,304.4267,58.1066,24.2737,22.9347,14.1641,30.4251,27.3885,18.1799


In [13]:
# Make distance integer
df['DIST'] = df['DIST'].apply(int)

In [14]:
df = df.loc[df['DIST']<=1000]
len(df)

93973

In [15]:
# Generate generalised cost
VoT = VoT.to_dict()
df['GC_RAIL'] = df['T_RAIL_Sc'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                   for c,d,p in zip(df['C_RAIL'], df['DIST'], df['PURPOSE'])]
df['GC_RAIL_S'] = df['T_RAIL_S_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                      for c,d,p in zip(df['C_RAIL_S'], df['DIST'], df['PURPOSE'])]
df['GC_RAIL_L'] = df['T_RAIL_L_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                      for c,d,p in zip(df['C_RAIL_L'], df['DIST'], df['PURPOSE'])]
df['GC_COACH'] = df['T_COACH_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                    for c,d,p in zip(df['C_COACH'], df['DIST'], df['PURPOSE'])]
df['GC_BUS'] = df['T_BUS_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                for c,d,p in zip(df['C_BUS'], df['DIST'], df['PURPOSE'])]
df['GC_ROAD'] = df['T_ROAD_S'] + [c / VoT[('Fz'+str(p), 'PT')][d]
                                  for c,d,p in zip(df['C_ROAD'], df['DIST'], df['PURPOSE'])]
df['GC_AIR'] = df['T_AIR_S'] + [c / VoT[('Fz'+str(p), 'air')][d]
                                for c,d,p in zip(df['C_AIR'], df['DIST'], df['PURPOSE'])]
df['GC_CAR'] = df['T_CAR_S'] + [c / VoT[('Fz'+str(p), 'car')][d]
                                for c,d,p in zip(df['C_CAR'], df['DIST'], df['PURPOSE'])]
df['GC_NON_MOTOR'] = df['T_NON_MOTOR_S']

In [16]:
# Create the initial database and make columns global variables
database = db.Database('MiD2017', df.copy())
globals().update(database.variables)
database.getSampleSize()

93973

### Estimation parameters

In [17]:
asc_rail = ex.Beta('asc_rail', 0, None, None, 1)
asc_rail_s = ex.Beta('asc_rail_s', 0, None, None, 0)
asc_rail_l = ex.Beta('asc_rail_l', 0, None, None, 0)
asc_coach = ex.Beta('asc_coach', 0, None, None, 0)
asc_bus = ex.Beta('asc_bus', 0, None, None, 0)
asc_road = ex.Beta('asc_road', 0, None, None, 0)
asc_air = ex.Beta('asc_air', 0, None, None, 0)
asc_car = ex.Beta('asc_car', 0, None, None, 0)
asc_non_motor = ex.Beta('asc_non_motor', 0, None, None, 0)

In [18]:
b_t = ex.Beta('b_t', 0, None, None, 0)
b_c = ex.Beta('b_c', 0, None, None, 0)
b_ac = ex.Beta('b_ac', 0, None, None, 0)

In [20]:
# non-linear time component
b_t2 = ex.Beta('b_t2', 0, None, None, 0)

In [19]:
# generalised cost function parameter
b_gc = ex.Beta('b_gc', 0, None, None, 0)

In [20]:
# Parameters for the nested logit structure
mu_pt = ex.Beta('mu_pt', 1, 1, 10, 0)

### Utility functions

In [23]:
'''# Aggregated formulation
V_RAIL = asc_rail + b_t * T_RAIL_S + b_c * C_RAIL + b_ac * AC_RAIL
V_COACH = asc_coach + b_t * T_COACH_S + b_c * C_COACH + b_ac * AC_COACH
V_BUS = asc_bus + b_t * T_BUS_S + b_c * C_BUS + b_ac * AC_BUS
V_AIR = asc_air + b_t * T_AIR_S + b_c * C_AIR + b_ac * AC_AIR
V_CAR = asc_car + b_t * T_CAR_S + b_c * C_CAR
V_NON_MOTOR = asc_non_motor + b_t * T_NON_MOTOR_S

# Aggregated formulation with non-linear perception of travel time
V_RAIL = asc_rail + b_t * T_RAIL + b_t2 * T_RAIL_S + b_c * C_RAIL + b_ac * AC_RAIL
V_COACH = asc_coach + b_t * T_COACH + b_t2 * T_COACH_S + b_c * C_COACH + b_ac * AC_COACH
V_BUS = asc_bus + b_t * T_BUS + b_t2 * T_BUS_S + b_c * C_BUS + b_ac * AC_BUS
V_AIR = asc_air + b_t * T_AIR + b_t2 * T_AIR_S + b_c * C_AIR + b_ac * AC_AIR
V_CAR = asc_car + b_t * T_CAR + b_t2 * T_CAR_S + b_c * C_CAR
V_NON_MOTOR = asc_non_motor + b_t * T_NON_MOTOR + b_t2 * T_NON_MOTOR_S'''
print('Deprected')

Deprected


In [21]:
# The cost damping function
def spline(x, beta=b_gc, Q=3, c0=0, c1=20, c2=40, c3=np.inf):
    alpha = [0, -beta/2*ex.Power(ex.log(c1),3),
             -beta/2*ex.log(c1)*(3*ex.Power(ex.log(c2),2)+ex.Power(ex.log(c1),2))] # for Q=3
    theta = [1, 3/2*ex.log(c1), 3*ex.log(c1)*ex.log(c2)] # for Q=3
    def component(x, q):
        return beta*theta[q-1]*ex.Power(ex.log(x),Q-q+1) + alpha[q-1]
    if x < c1:
        return component(x,1)
    elif x < c2:
        return component(x,2)
    elif x >= c2:
        return component(x,3)

In [22]:
# Aggregated formulation with damped generalised cost
V_RAIL = asc_rail + spline(GC_RAIL)
V_RAIL_S = asc_rail_s + spline(GC_RAIL_S)# + b_ac * AC_RAIL
V_RAIL_L = asc_rail_l + spline(GC_RAIL_L)# + b_ac * AC_RAIL
V_COACH = asc_coach + spline(GC_COACH)# + b_ac * AC_COACH
V_BUS = asc_bus + spline(GC_BUS)# + b_ac * AC_BUS
V_ROAD = asc_road + spline(GC_ROAD)
V_AIR = asc_air + spline(GC_AIR)# + b_ac * AC_AIR
V_CAR = asc_car + spline(GC_CAR)
V_NON_MOTOR = asc_non_motor + spline(GC_NON_MOTOR)

### Run the estimation

In [23]:
# Define level of verbosity
logger = message.bioMessage()
#logger.setSilent()
logger.setWarning()
#logger.setGeneral()
#logger.setDetailed()

In [24]:
# Map modes to utility functions
V = {1:V_RAIL,
#     1:V_RAIL_S,
#     2:V_RAIL_L,
#     3:V_COACH,
     4:V_ROAD,#BUS,
     5:V_AIR,
     6:V_CAR,
     7:V_NON_MOTOR}

In [25]:
# Map the availability of alternatives with MODE as key
# PT is always available
av = {1:RAIL_AV,#RAIL_S_AV,
#      2:RAIL_L_AV,
#      3:COACH_AV,
      4:ROAD_AV,#BUS_AV,
      5:AIR_AV,
      6:CAR_AV,
      7:NON_MOTOR_AV}

In [26]:
# Mode nests as tuples with nest name and dictionary where
# alternative IDs are mapped to alpha values. Missing ID's alpha is zero
# Alternatively use lists with mode ID without alpha
pt = mu_pt, [1, 4]#[1, 2, 3, 4]
air = 1, [5]
car = 1, [6]
walk = 1, [7]
nests = pt, air, car, walk

In [27]:
# Write results to an Excel file
writer = pd.ExcelWriter(input_path + 'estimation_results_car_own.xls', engine='xlsxwriter')

In [28]:
# Choose the multinomial logit model
mnl = models.loglogit(V, av, MODE)

In [29]:
model_mnl = bio.BIOGEME(database, mnl)
model_mnl.modelName = 'MNL'

In [30]:
results_mnl = model_mnl.estimate()

In [31]:
results = results_mnl.getEstimatedParameters()
for key, val in results_mnl.getGeneralStatistics().items():
    results.loc[key] = [val[0], val[1]] + ['' for i in range(len(results.columns)-2)]
results

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_air,-6.597812,0.169457,-38.9349,0,0.197767,-33.3615,0
asc_non_motor,-4.284559,0.0353802,-121.101,0,0.0350503,-122.24,0
asc_rail,-2.195415,0.0141215,-155.466,0,0.0160893,-136.452,0
asc_road,-2.995701,0.0183046,-163.658,0,0.0192166,-155.892,0
b_gc,-0.360508,0.00633913,-56.8703,0,0.0129954,-27.7412,0
Number of estimated parameters,5.000000,,,,,,
Sample size,93973.000000,,,,,,
Excluded observations,0.000000,,,,,,
Init log likelihood,-128830.230708,.7g,,,,,
Final log likelihood,-38666.958176,.7g,,,,,


In [32]:
results.to_excel(writer, sheet_name=model_mnl.modelName)

In [33]:
# Choose the logarithmic nested logit model
nl = models.lognested(V, av, nests, MODE)

In [34]:
# Nested Logit
model_nl = bio.BIOGEME(database, nl)
model_nl.modelName = 'NL'

In [35]:
results_nl = model_nl.estimate()

In [36]:
results = results_nl.getEstimatedParameters()
for key, val in results_nl.getGeneralStatistics().items():
    results.loc[key] = [val[0], val[1]] + ['' for i in range(len(results.columns)-2)]
results

,Value,Active bound,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
asc_air,-6.591381,0,0.237263,-27.7809,0,1.04735,-6.29338,3.10635e-10
asc_non_motor,-4.284703,0,0.0431881,-99.2103,0,0.154078,-27.8086,0
asc_rail,-2.195449,0,0.0320828,-68.4308,0,0.185524,-11.8338,0
asc_road,-2.995698,0,0.114187,-26.235,0,0.71585,-4.18481,2.854e-05
b_gc,-0.360454,0,0.0228412,-15.7809,0,0.135556,-2.65907,0.0078356
mu_pt,1.000000,1,0.122102,8.18987,2.22045e-16,0.774101,1.29182,0.196419
Number of estimated parameters,6.000000,,,,,,,
Sample size,93973.000000,,,,,,,
Excluded observations,0.000000,,,,,,,
Init log likelihood,-128830.230708,.7g,,,,,,


In [37]:
results.to_excel(writer, sheet_name=model_nl.modelName)

In [38]:
# Run the estimation by purpose
results = []
for p in [1,2,3,4,6] for car_ownership in [0,1]:
    database = db.Database('MiD2017', df.copy())
    database.remove(PURPOSE!=p)
    database.remove(CAR_AV!=car_ownership)
    print('Sample size for purpose {}, car_ownership {}: {}'.format(
        p, car_ownership, database.getSampleSize()))
    model = bio.BIOGEME(database, mnl) # Choose the model formulation
    model.modelName = 'MNL_Fz' + str(p) + '_' + str(car_ownership) # Name it
    results.append(model.estimate()) # Estimation
    output = results[-1].getEstimatedParameters()
    # Add results to the Excel file
    for key, val in results[-1].getGeneralStatistics().items():
        output.loc[key] = [val[0], val[1]] + ['' for i in range(len(output.columns)-2)]
    output.to_excel(writer, sheet_name=model.modelName)

Sample size for purpose 1: 25317
Sample size for purpose 2: 3638
Sample size for purpose 3: 23047
Sample size for purpose 4: 5620
Sample size for purpose 6: 36351


In [39]:
writer.save()

In [40]:
# Generate LaTeX code
file = open(input_path + 'estimation_results_LaTeX_code.txt', 'w')
for r in results:
    file.write(r.getLaTeX())
file.close()